In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Koshurnikovo,RU,2022-03-12 23:16:47,54.1667,93.3000,-17.00,98,32,3.38
1,1,Dzhebariki-Khaya,RU,2022-03-12 23:16:48,62.2167,135.8000,-11.42,100,100,2.15
2,2,Santa Rosa,PH,2022-03-12 23:15:19,14.3122,121.1114,78.78,92,99,3.51
3,3,Talant,FR,2022-03-12 23:16:49,47.3369,5.0089,47.41,87,0,0.00
4,4,Fortuna,US,2022-03-12 23:15:19,40.5982,-124.1573,57.54,64,100,17.27


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [13]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Santa Rosa,PH,2022-03-12 23:15:19,14.3122,121.1114,78.78,92,99,3.51
10,10,Kismayo,SO,2022-03-12 23:16:52,-0.3582,42.5454,77.92,81,14,14.58
11,11,Cabo San Lucas,MX,2022-03-12 23:13:44,22.8909,-109.9124,77.61,46,0,13.80
19,19,Monteiro,BR,2022-03-12 23:16:56,-7.8894,-37.1200,80.94,46,96,8.52
20,20,Vila Velha,BR,2022-03-12 23:16:57,-20.3297,-40.2925,80.55,88,75,5.75
23,23,East London,ZA,2022-03-12 23:16:58,-33.0153,27.9116,78.15,75,73,11.59
26,26,Kapaa,US,2022-03-12 23:17:00,22.0752,-159.3190,82.38,67,20,12.66
30,30,Dingle,PH,2022-03-12 23:17:02,10.9995,122.6711,77.65,88,60,4.29
31,31,Rikitea,PF,2022-03-12 23:17:03,-23.1203,-134.9692,79.70,71,64,14.00
32,32,Georgetown,MY,2022-03-12 23:16:07,5.4112,100.3354,81.95,92,20,8.05


In [17]:
preferred_cities_df.count()

City_ID       158
City          158
Country       158
Date          158
Lat           158
Lng           158
Max Temp      158
Humidity      158
Cloudiness    158
Wind Speed    158
dtype: int64

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Santa Rosa,PH,78.78,14.3122,121.1114,
10,Kismayo,SO,77.92,-0.3582,42.5454,
11,Cabo San Lucas,MX,77.61,22.8909,-109.9124,
19,Monteiro,BR,80.94,-7.8894,-37.1200,
20,Vila Velha,BR,80.55,-20.3297,-40.2925,
23,East London,ZA,78.15,-33.0153,27.9116,
26,Kapaa,US,82.38,22.0752,-159.3190,
30,Dingle,PH,77.65,10.9995,122.6711,
31,Rikitea,PF,79.70,-23.1203,-134.9692,
32,Georgetown,MY,81.95,5.4112,100.3354,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [30]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [39]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))